## Introduction
This notebook is designed to guide you through the process of building a simple retrieval augmented generation application using Langchain, ChromaDB and OpenAI's GPT-4 LLM. 

### RAG: Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) combines the power of language models with external content retrieval to answer questions based on specific content. This approach allows for more accurate and contextually relevant responses by leveraging a database of information.

The process involves several key steps:
- **Content Retrieval and Storage**: First, we ingest and store our content in a searchable format. This involves fetching content from a URL, splitting it into manageable parts, embedding these parts for semantic search, and storing them in a Vector DB.
- **Question Answering**: When a user question is posed, the system retrieves relevant content from the Vector DB and uses it to generate an answer with the LLM.

This notebook will walk you through setting up a RAG system using ChromaDB for content storage and retrieval, and OpenAI's GPT-4 for question answering.

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import bs4

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split the content into manageable chunks for better retrieval.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed the chunks and store them in ChromaDB for efficient retrieval.
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [3]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Set up the RAG chain for retrieving and generating answers.
retriever = vectorstore.as_retriever()
system_prompt = ("""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Context: {context}
""")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)
                 

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

model = ChatOpenAI(model="gpt-4o")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [4]:
question = "What is tool usage?"
rag_chain.invoke(question)

"Tool usage refers to the creation, modification, and utilization of external objects to perform tasks that go beyond an individual's physical and cognitive limits. It is a distinguishing characteristic of human beings and can significantly extend the capabilities of systems, such as equipping language models with external tools to enhance their functionality."